### Import Relevant Libraries

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import seaborn as sns
sns.set()

### Generate Master DF

In [2]:
master_df = pd.read_csv('Consumer_Complaints.csv')

/Users/seanmcalevey/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocessing: dropping missing narratives, removing 'X's, and taking an even sample of 1,000(/10,000) narratives with timely and un-timely responses

In [3]:
proc_df = master_df.dropna(subset=['Consumer complaint narrative'])

In [4]:
#proc_df['Rapid response'] = (proc_df['Timely response?'] == 'Yes') & (proc_df['Consumer disputed?'] != 'Yes')

In [5]:
#proc_df['Rapid response'].value_counts()

In [6]:
new_df = proc_df[proc_df['Timely response?']=='Yes'].sample(10000)

new_df2 = proc_df[proc_df['Timely response?']=='No'].sample(10000)

df = new_df.append(new_df2)

In [73]:
first_clean = [re.sub('[^A-WY-Za-z0-9]+', ' ', nar) for nar in df['Consumer complaint narrative']]

#second_clean = [re.sub("\'", ' ', nar) for nar in first_clean]

df['Cleaned narratives'] = [nar for nar in first_clean]

In [74]:
from nltk import word_tokenize, pos_tag

def nouns_adjs(text):
    
    # So far it seems like the best POS tags are: Nouns, Adjectives, Adverbs, Superlative Adjs
    
    """Noun 'NN', Adjective 'JJ', Adverb 'RB', Superlative Adverb 'RBS', Superlative Adjective 'JJS', 
    Personal Pronoun 'PRP', Personal Pronoun Possessive 'PRP$' """
    
    noun_adj = lambda pos: (pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'RB' or pos[:2] == 'RBS'
                            or pos[:2] == 'JJS')
    tokenized = word_tokenize(text)
    nouns_adjs = [word for (word, pos) in pos_tag(tokenized) if noun_adj(pos)]
    output = ' '.join(nouns_adjs)
    
    return output

In [76]:
df['Cleaned narratives'] = [nouns_adjs(nar) for nar in df['Cleaned narratives']]

In [77]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Cleaned narratives
29655,08/15/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,The below credit check happened on my account ...,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,NJ,08901,NaN,Consent provided,Web,08/15/2019,Closed with non-monetary relief,Yes,NaN,3342824,below credit check account not ever bank apply...
664983,02/22/2017,Bank account or service,Savings account,"Account opening, closing, or management",NaN,"Consumer Finance, I contacted Navy Federal Cre...",Company believes it acted appropriately as aut...,NAVY FEDERAL CREDIT UNION,VA,221XX,NaN,Consent provided,Web,02/22/2017,Closed with explanation,Yes,Yes,2355148,Consumer Finance Navy Federal Credit Union NFC...
470797,12/09/2017,"Money transfer, virtual currency, or money ser...",Virtual currency,Other transaction problem,NaN,The situation is detailed below. \n\nOn XXXX X...,NaN,"Coinbase, Inc.",IL,NaN,NaN,Consent provided,Web,12/09/2017,Closed with explanation,Yes,NaN,2750627,situation money bank account Bank conversation...
919626,10/27/2015,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,The debt is from XX/XX/XXXX-XX/XX/XXXX past st...,NaN,"Columbia Collection Service, Inc.",OR,970XX,NaN,Consent provided,Web,10/27/2015,Closed with explanation,Yes,Yes,1626270,debt past statute never bill now current month...
245472,10/20/2018,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I GOT A LOAN FROM ONE MAIN AND THEY PULLED MY ...,NaN,"ONEMAIN FINANCIAL HOLDINGS, LLC.",IL,604XX,NaN,Consent provided,Web,10/20/2018,Closed with non-monetary relief,Yes,NaN,3051771,LOAN FROM ONE MAIN AND THEY PULLED MY CREDIT T...


In [78]:
# from textblob import TextBlob

# df['Subjectivity'] = [TextBlob(blob).subjectivity for blob in df['Cleaned narratives'] ]

# df['Polarity'] = [TextBlob(blob).polarity for blob in df['Cleaned narratives']]

### Train_test_split the data

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Cleaned narratives'], 
                                                       df['Timely response?'], random_state=42)

### Tfidf Vectorize

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

#vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,3), min_df=5)

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=3)

X_train = vectorizer.fit_transform(X_train)

X_test = vectorizer.transform(X_test)

# X_train['Cleaned narratives'] = vectorizer.fit_transform(X_train['Cleaned narratives'])

# X_test['Cleaned narratives'] = vectorizer.transform(X_test['Cleaned narratives'])

In [86]:
# from sklearn.decomposition import TruncatedSVD

# n_components = 1000

# svd = TruncatedSVD(n_components)
    
# X_train_nar = svd.fit_transform(X_train)

# X_test_nar = svd.transform(X_test)
    
# explained_variance_1 = round(100*sum(svd.explained_variance_ratio_), 3)

# print('Retained info after SVD: ' + str(explained_variance_1) + '%')

In [87]:
# X_train = np.concatenate((X_train_nar, np.asarray(X_train[['Subjectivity', 'Polarity']])), axis=1)

# X_test = np.concatenate((X_test_nar, np.asarray(X_test[['Subjectivity', 'Polarity']])), axis=1)

### Support Vector Classifier

In [88]:
from sklearn.svm import SVC

svc_pipe = make_pipeline(SVC(kernel='linear'))

param_grid = {'svc__C': [0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 1, 2, 5]}

cv_input = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

grid = GridSearchCV(svc_pipe, param_grid, cv=cv_input).fit(X_train, y_train)

print(f'Best param: {grid.best_params_}')

print(f'Train score: {grid.score(X_train, y_train)}')

print(f'Test score: {grid.score(X_test, y_test)}')

Best param: {'svc__C': 0.4}
Train score: 0.8243333333333334
Test score: 0.68


### Random Forest Classifier

In [40]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=500)

rf_clf.fit(X_train, y_train)

print(f'Train score: {rf_clf.score(X_train, y_train)}')

print(f'Test score: {rf_clf.score(X_test, y_test)}')

Train score: 0.9993333333333333
Test score: 0.702


### Gradient Boosting Classifier

In [41]:
from sklearn.ensemble import GradientBoostingClassifier

pipe = make_pipeline(GradientBoostingClassifier(n_estimators=500))

param_grid = {'gradientboostingclassifier__learning_rate': [0.025, 0.05, 0.075, 0.1, 0.2, 0.5]}

cv_input = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, cv=cv_input).fit(X_train, y_train)

print(f'Best params: {grid.best_params_}')

print(f'Train score: {grid.score(X_train, y_train)}')

print(f'Test score: {grid.score(X_test, y_test)}')

Best params: {'gradientboostingclassifier__learning_rate': 0.075}
Train score: 0.767
Test score: 0.684


### Bernoulli Naive Bayes Classifier

In [42]:
from sklearn.naive_bayes import BernoulliNB

pipe = make_pipeline(BernoulliNB())

param_grid = {'bernoullinb__alpha': [0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8]}

cv_input = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, cv=cv_input).fit(X_train, y_train)

#bernoulli.fit(X_train, y_train)

print(f'Best params: {grid.best_params_}')

print(f'Train score: {grid.score(X_train, y_train)}')

print(f'Test score: {grid.score(X_test, y_test)}')

Best params: {'bernoullinb__alpha': 0.8}
Train score: 0.855
Test score: 0.703


### AdaBoost Classifier

In [44]:
from sklearn.ensemble import AdaBoostClassifier

pipe = make_pipeline(AdaBoostClassifier(n_estimators=500))

param_grid = {'adaboostclassifier__learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.5]}

cv_input = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, cv=cv_input).fit(X_train, y_train)

print(f'Best params: {grid.best_params_}')

print(f'Train score: {grid.score(X_train, y_train)}')

print(f'Test score: {grid.score(X_test, y_test)}')

Best params: {'adaboostclassifier__learning_rate': 0.07}
Train score: 0.7273333333333334
Test score: 0.682


### XGB Classifier

In [45]:
from xgboost import XGBClassifier

pipe = make_pipeline(XGBClassifier())

param_grid = {'xgbclassifier__eta': [0.01, 0.025, 0.05, 0.075, 0.1, 0.2]}

cv_input = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

grid = GridSearchCV(pipe, param_grid, cv=cv_input).fit(X_train, y_train)

print(f'Best params: {grid.best_params_}')

print(f'Train score: {grid.score(X_train, y_train)}')

print(f'Test score: {grid.score(X_test, y_test)}')

Best params: {'xgbclassifier__eta': 0.05}
Train score: 0.7736666666666666
Test score: 0.665


In [595]:
# import pickle

# filename = 'XGBClassifier'

# outfile = open(filename, 'wb')

In [596]:
# pickle.dump(grid, outfile)

# outfile.close()